In [18]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [19]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.Defaulting to user installation because normal site-packages is not writeable



In [20]:
pip install requests

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install transformers

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [22]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [23]:
pip install nltk

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [24]:
pip install rouge_score

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [25]:
pip install evaluate

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [26]:
pip install bert_score

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [27]:
pip install torch

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [28]:
pip install numpy

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [1]:
import requests
from bs4 import BeautifulSoup
import transformers
from transformers import pipeline
pipe = pipeline("summarization", model="facebook/bart-large-cnn")

C:\Users\eolor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OSError: [WinError 126] The specified module could not be found. Error loading "C:\Users\eolor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\lib\shm.dll" or one of its dependencies.

In [ ]:
from evaluate import load
import evaluate

In [2]:
import transformers
from transformers import BertTokenizer, BertForMaskedLM, BertModel, pipeline
import bert_score
from bert_score import BERTScorer

OSError: [WinError 126] The specified module could not be found. Error loading "C:\Users\eolor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\lib\shm.dll" or one of its dependencies.

In [ ]:
import re
import pandas as pd

In [ ]:
def get_data(url):
    """ Retrieves text surrounded by <p> tags from the url

        Args:
          url: A url passed in by the user, containing the text to be summarised

        Returns:
          data: An array of each paragraph from the url
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    data = soup.find_all('p')
    return data

In [ ]:
#text = str (text)
#text

In [ ]:
def clean_data(text):
  """ Cleans the data so it can be processed by the pipeline

  Args:
    text: raw data from the get_data() method

  Returns:
    paragraph: An array containing all the clean data, separated by paragraph
  """
  paragraph = []
  for i in text:
    paraString = str (i)
    match = re.match("<p>(.*)<\/p>", paraString)
    if match:
      groupMatch = re.match("^(?!.*<[^>]+>).*$", match.group(1))
      if groupMatch:
        paragraph.append(match.group(1))
  return paragraph

In [ ]:
def get_summary(text, maxLim, minLim, sample):
  """ Summarizes the text paragraph by paragraph

  Args:
    text: An array of clean text(s)
    max: An integer stating the maximum amount of words the summary should include
    min: An integer stating amount of words the summary should include
    sample: A boolean that influences how the model generates text

  Returns:
    summary: A dictionary full of key-value pairs ('summary_text': summarised paragraph)
  """
  summary = pipe(text, max_length=maxLim, min_length=minLim, do_sample=sample)
  return summary

In [ ]:
def get_dataframe(para, summ):
  """ Turns the original paragraphs and summarised paragraphs into a Data Frame

    Args:
      para: The original cleaned paragraphs
      summ: The summary dictionary

    Returns:
      textDF: Data Frame containing both values of para and summ
  """
  summaries = []
  for i in range(len(summ)):
    summaries.append(summ[i]['summary_text'])
  para = pd.DataFrame(para, columns=['Paragraph'])
  summaries = pd.DataFrame(summaries, columns=['Summary'])
  textDF = pd.concat([para, summaries], axis=1)
  return textDF

In [ ]:
text = get_data('https://www.forbes.com/sites/forbestechcouncil/2023/04/10/the-future-of-artificial-intelligence/?sh=6043be4ac496')
text

In [ ]:
cleanText = clean_data(text)

In [ ]:
summary = get_summary(cleanText, 35, 25, False)

In [ ]:
summaryDF = get_dataframe(cleanText, summary)
summaryDF

In [ ]:
rouge = evaluate.load('rouge')
candidates = summaryDF['Summary']
references = summaryDF['Paragraph']
results = rouge.compute(predictions=candidates, references=references)
results

{'rouge1': 0.6800088764767979,
 'rouge2': 0.6606512550545043,
 'rougeL': 0.6053253597173891,
 'rougeLsum': 0.604088330604371}

In [ ]:
import torch
import numpy as np

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

text1 = summaryDF['Paragraph'].tolist()
text2 = summaryDF['Summary'].tolist()

inputs1 = tokenizer(text1, return_tensors="pt", padding=True, truncation=True)
inputs2 = tokenizer(text2, return_tensors="pt", padding=True, truncation=True)

outputs1 = model(**inputs1)
outputs2 = model(**inputs2)

embeddings1 = outputs1.last_hidden_state.mean(dim=1).detach().numpy()
embeddings2 = outputs2.last_hidden_state.mean(dim=1).detach().numpy()

similarity = np.dot(embeddings1, embeddings2.T) / (np.linalg.norm(embeddings1) * np.linalg.norm(embeddings2))

results = "Similarity between the texts: {:.4f}".format(similarity[0][0])
results

'Similarity between the texts: 0.0602'